In [1]:
using ITensors, ITensorMPS
using IterTools
using Plots
using JSON3

include("simulation/mpo_simulation.jl")
using .MPOSimulation
include("utils/utils.jl")
using .Utils
include("simulation/mps_simulation.jl")
using .MPSSimulation
include("simulation/reconstruct_expectation.jl")
using .ReconstructExpectation

In [ ]:
using Printf
using Plots

function print_and_plot(results::Vector{Results})
    print_results(results)

    sorted_results = sort(results, by = r -> r.time)
    times = [r.time for r in sorted_results]
    
    p1 = plot(times, 
        [r.expval_true for r in sorted_results],
        label="True expval",
        marker=:circle,
        markersize=4,
        linewidth=2,
        title="Expectation Values vs Time",
        xlabel="Time (s)",
        ylabel="Expectation Value"
    )
    
    plot!(p1, 
        times, 
        [r.expval_tensors for r in sorted_results],
        label="Tensors",
        marker=:square,
        markersize=4,
        linewidth=2,
        legend=:outerbottom
    )
    
    p2 = plot(times,
        [r.maxdim for r in sorted_results],
        marker=:circle,
        markersize=4,
        linewidth=2,
        label="MaxDim",
        title="Maximum Dimension vs Time",
        xlabel="Time (s)",
        ylabel="Maximum Dimension",
        legend=:outerbottom
    )
    
    plot(p1, p2, layout=(1,2), size=(1200,400))
end

function print_results(results::Vector{Results})

    println("\n============= Results Summary =============")
    println(
        rpad("Time (s)", 12) *
        rpad("True expval", 15) *
        rpad("Tensors", 15) *
        rpad("Discrepancy", 15) *
        "MaxDim"
    )
    println("─"^65)  

    for r in results
        println(
            rpad(@sprintf("%.3f", r.time), 12) *
            rpad(@sprintf("%.6f", r.expval_true), 15) *
            rpad(@sprintf("%.6f", r.expval_tensors), 15) *
            rpad(@sprintf("%.2e", r.discrepancy), 15) *
            string(r.maxdim)
        )
    end
    
end

In [ ]:
# Simulation using MPS - multiple timesteps
basis_gates = ["h", "rx", "ry", "rz", "rxx", "rzz", "ryy", "cx"]
cutoff = 1e-12
maxdim = 1000
method = "densitymatrix" # Currently not used


subcircuits_dir = "subcircuits/"
coeffs_expvals_dir = "expvals/"
expval_results = Results[]


file_pairs = pair_files(subcircuits_dir, coeffs_expvals_dir)
for (subcircuit_json, coeffs_expvals_json, t) in file_pairs

    circuits_data, expval_true, coefficients, num_qubits_impurity = parse_subcircuits(subcircuit_json);

    coeffs_expvals_data = open(coeffs_expvals_json, "r") do io
        JSON3.read(io, Vector{CoeffExpPair})
    end

    all_mpo_sequences = build_all_mpo_sequences(circuits_data, basis_gates);
    all_psis = apply_mpo_sequences(circuits_data, all_mpo_sequences, cutoff, maxdim, method);
    

    labels = collect(keys(all_psis))
    sites_per_label = Dict{String, Vector{Index{Int64}}}()
    for label in labels
        sites_per_label[label] = siteinds(all_psis[label][1][1][1])
    end
    
    # Generate union of sites
    sites_union = vcat(union(values(sites_per_label))...)
    # Create the observables
    # Careful with ordering!

    # for measuring ZZZ on impurity qubits
    # obs = [MPO(sites_union, "Z")] 
    obs = vcat([create_mpo("Id", 1, sites_union)])
    # for measuring ZII + IZI + IIZ on all qubits
    # obs = vcat([create_mpo("Id", i, sites_union) for i in 1:num_qubits_impurity], [create_mpo("Z", i, sites_union) for i in 1:length(sites_union)])

    # for measuring ZII + IZI + IIZ on the impurity qubits 
    # obs = vcat([create_mpo("Z", i, sites_union) for i in 1:length(sites_union)])
    
    # for measuring ZII + IZI + IIZ on the impurity qubits (when working on all qubits)
    # obs = vcat([create_mpo("Z", i, sites_union) for i in 1:5], [create_mpo("Id", 1, sites_union) for i in 1:8])
    
    # for measuring ZII + IZI + IIZ on the bath qubits (when working on the impurity qubits)
    # obs = vcat([create_mpo("Id", 1, sites_union) for i in 1:num_qubits_impurity])
     
        
    coef_temp_pairs = CoeffExpPair[]

    for (k, coeff) in enumerate(coefficients)
        expval_list = zeros(length(obs))  
        
        combinations = IterTools.product([all_psis[label][k] for label in labels]...)
        
        for combination in combinations
            mps_dict = Dict{String, MPS}()
            prob = 1.0
            for (idx, label) in enumerate(labels)
                psi_label, p_label = combination[idx]
                mps_dict[label] = psi_label
                prob *= p_label
            end
            psi = mps_tensor_product(mps_dict, sites_union)
            
            for (i, observable) in enumerate(obs)
                expval = compute_expval(psi, observable)
                expval_list[i] += prob * expval
            end
        end
        
        push!(coef_temp_pairs, CoeffExpPair(coeff, expval_list))
    end
    
    expval_tensors = compute_expval_knitting(coeffs_expvals_data, coef_temp_pairs)
    
    
    discrepancy = abs(round(100 * (expval_true - expval_tensors) / expval_true, digits=4))

    max_dim = -Inf
    for label in labels
        for psis in all_psis[label]
            for psi in psis
                mps = psi[1]
                current_dim = maxlinkdim(mps) 
                if current_dim > max_dim
                    max_dim = current_dim
                end
            end
        end
    end
    print(expval_true, " ", expval_tensors, " ", t, " ", max_dim, "\n")
    push!(expval_results, Results(t, expval_true, expval_tensors, discrepancy, max_dim))   
end

output_dir = "expvals_final_julia/"
JSON3.write(joinpath(output_dir, "expval_results_fullcirc_impurityexpval.json"), expval_results)
println("Results have been written to $(joinpath(output_dir, "expval_results.json"))")

print_and_plot(expval_results)

In [5]:
# TODO

#compute number of parameters using mps formula
# implement real unitary
# XXX 3. Unify Qiskit and Julia code (I think this is done now!)
# 4. I also want to analyze bound dimension by just looking (after knitting) at both subsystems. SO i have to check if I can pass the impurity to julia
